In [ ]:
import tensorflow as tf

# Create tensors
tensor1 = tf.constant([1, 2, 3])
tensor2 = tf.constant([4, 5, 6])

# Add tensors
result_tensor = tf.add(tensor1, tensor2)

# Print result
print("Result tensor:", result_tensor)


In [ ]:
import numpy as np
import tensorflow as tf


In [ ]:
model_path = "model.tflite"

In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

In [ ]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input details
print("Input details:")
for detail in input_details:
    print(detail)

# Print output details
print("\nOutput details:")
for detail in output_details:
    print(detail)

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define class labels
class_labels = ['apple', 'banana', 'orange']

# Preprocess the image
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((input_details[0]['shape'][1], input_details[0]['shape'][2]))  # Resize to match the input shape of the model
    img = np.array(img) / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.astype(input_details[0]['dtype'])

# Path to the random image
image_path = "orange.jpg"

# Preprocess the image
preprocessed_img = preprocess_image(image_path)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], preprocessed_img)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Get predicted class index
predicted_class_index = np.argmax(output_data)
predicted_class_label = class_labels[predicted_class_index]

# Display the prediction
print("Predicted class:", predicted_class_label)


In [ ]:
output_data

In [ ]:
preprocessed_img.shape

### Train the model

In [ ]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.Rescaling(1./255),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Conv2D(32, 3, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.Dense(3)
    ]
)
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [ ]:
##new

import tensorflow as tf

# Define the model with input shape (32, 32, 3)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),  # Input shape modified here
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(3)
])

model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
img_height, img_width = 32, 32
batch_size = 20

train_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/train",
    image_size = (img_height, img_width),
    batch_size = batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/validation",
    image_size = (img_height, img_width),
    batch_size = batch_size
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/test",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

In [ ]:
class_names = ["apple", "banana", "orange"]
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
model.save('fruit')

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
  classifications = model(images)
  print(classifications)
  
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + " | Real: " + class_names[labels[i]])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('fruit')

# Step 1: Read the image file into a tensor
image_path = 'apple.jpg'  # Replace this with the path to your image file
image_data = tf.io.read_file(image_path)

# Step 2: Decode the image data into a tensor
image_tensor = tf.image.decode_jpeg(image_data, channels=3)

# Step 3: Resize the image to 32x32x3
image_tensor_resized = tf.image.resize(image_tensor, [32, 32])

# Step 4: Add a batch dimension
image_tensor_batch = tf.expand_dims(image_tensor_resized, axis=0)

output = model(image_tensor_batch)
output

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("fruits.tflite", 'wb') as f:
  f.write(tflite_model)

### tflite inference

In [ ]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='fruits.tflite')
interpreter.allocate_tensors()


In [ ]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
import numpy as np
from PIL import Image

# Preprocess the image
def preprocess_image(image_path, input_shape):
    img = Image.open(image_path)
    img = img.resize((input_shape[1], input_shape[2]))  # Resize to match the input shape of the model
    img = np.array(img) / 255.0  # Normalize the image
    img = img.astype(input_details[0]['dtype'])  # Convert to the required data type
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Define the input shape (adjust according to your model)
input_shape = input_details[0]['shape']

# Path to the input image
image_path = r'apple.jpg'

# Preprocess the image
input_data = preprocess_image(image_path, input_shape)

plt.imshow(input_data[0])
plt.axis('off')  # Hide axis
plt.show()



In [ ]:
input_data.shape

In [ ]:
input_details[0]

In [ ]:
# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

In [ ]:
output_details[0]['index']

In [ ]:
# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Post-process the output (if needed)
predicted_class_index = np.argmax(output_data)

In [ ]:
predicted_class_index

In [ ]:
output_data

In [ ]:
# Display the predicted class label
class_labels = ['Apple', 'Banana', 'Orange']  # Define your class labels
predicted_class_label = class_labels[predicted_class_index]
print("Predicted class:", predicted_class_label)

In [ ]:
import tensorflow as tf
import numpy as np

# Step 1: Load the image using TensorFlow
image_path = 'orange.jpg'  # Replace this with the path to your image file
image_data = tf.io.read_file(image_path)
image_tensor = tf.image.decode_jpeg(image_data, channels=3)

# Step 2: Preprocess the image (resize, normalize, etc.)
image_tensor = tf.image.resize(image_tensor, [32, 32])  # Resize the image to 32x32
# image_tensor = image_tensor / 255.0  # Normalize the pixel values to [0, 1]/

# Step 3: Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='fruits.tflite')
interpreter.allocate_tensors()

# Step 4: Run inference on the loaded image
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Ensure the input tensor shape and type matches the model input
input_shape = input_details[0]['shape']
input_data = np.expand_dims(image_tensor, axis=0).astype(input_details[0]['dtype'])

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Output data contains the inference result
output_data

In [ ]:
output_data

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='fruits.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Step 1: Read the image file into a tensor
image_path = 'orange.jpg'  # Replace this with the path to your image file
image_data = tf.io.read_file(image_path)

# Step 2: Decode the image data into a tensor
image_tensor = tf.image.decode_jpeg(image_data, channels=3)

# Step 3: Resize the image to 32x32x3
image_tensor_resized = tf.image.resize(image_tensor, [32, 32])

# Step 4: Normalize the image data (if necessary)
# Depending on how the model was trained, you may need to normalize the image data

# Preprocess the image tensor according to the input tensor details
input_shape = input_details[0]['shape']
input_data = tf.cast(image_tensor_resized, dtype=np.float32)
input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension if necessary

# Set the input tensor values
interpreter.set_tensor(input_details[0]['index'], input_data)

# Perform inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
